## [A Practitioner's Guide to Natural Language Processing](https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72)

### Table of Contents:

1. Data Retrieval with Web Scraping
2. Text wrangling and pre-processing
3. Parts of Speech Tagging
4. Shallow Parsing
5. Constituency and Dependency Parsing
6. Named Entity Recognition
7. Emotion and Sentiment Analysis

## Standard NLP Workflow

`CRISP-DM model` - an industry standard for executing any data science project.

Any NLP-based problem can be solved by a methodical workflow that has a sequence of steps:

![title](media/nlp-workflow.png)

## 1. Scraping News Articles for Data Retrieval

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

In [4]:
seed_urls = ['https://inshorts.com/en/read/technology',
             'https://inshorts.com/en/read/sports',
             'https://inshorts.com/en/read/world']

def build_dataset(seed_urls):
    news_data = []
    for url in seed_urls:
        news_category = url.split('/')[-1]
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'html.parser')
        
        news_articles = [{'news_headline': headline.find('span', 
                                                         attrs={"itemprop": "headline"}).string,
                          'news_article': article.find('div', 
                                                       attrs={"itemprop": "articleBody"}).string,
                          'news_category': news_category}
                         
                            for headline, article in 
                             zip(soup.find_all('div', 
                                               class_=["news-card-title news-right-box"]),
                                 soup.find_all('div', 
                                               class_=["news-card-content news-right-box"]))
                        ]
        news_data.extend(news_articles)
        
    df =  pd.DataFrame(news_data)
    df = df[['news_headline', 'news_article', 'news_category']]
    return df

In [5]:
news_df = build_dataset(seed_urls)
news_df.head(10)

,news_headline,news_article,news_category
0,"Xiaomi Redmi K20 Pro launched in India, to go ...",Xiaomi's flagship Redmi K20 Pro has been unvei...,technology
1,New TVS Ntorq 125 scooter features Bluetooth &...,"The TVS Ntorq 125, India's first scooter comes...",technology
2,"Chrome, Firefox users' data leaked, for sale a...",Extensions on Chrome and Firefox internet brow...,technology
3,Privacy concerns raised over photo app FaceApp...,The Russian company behind the viral photo-edi...,technology
4,Bluetooth flaw exposes user locations; Android...,Boston University researchers have found a fla...,technology
5,Twitter gets trolled after unveiling redesigne...,Micro-blogging platform Twitter was trolled on...,technology
6,Musk's Neuralink unveils 'threads' that link b...,Elon Musk has unveiled his new startup Neurali...,technology
7,Good: Elizabeth Warren on Peter Thiel saying s...,US politician Elizabeth Warren shared an artic...,technology
8,"Microsoft posts record Q4 results, shares hit ...",Microsoft posted its Q4 FY19 earnings report r...,technology
9,Google may be fined ₹136cr for abuse of Androi...,India antitrust watchdog Competition Commissio...,technology


In [6]:
news_df.news_category.value_counts()

world         25
technology    24
sports        24
Name: news_category, dtype: int64

## 2. Text Wrangling && Pre-processing

In [15]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata
nlp = spacy.load('en', parse=True, tag=True, entity=True)
#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

### Remove HTML tags

In [16]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

strip_html_tags('<html><h2>Some important text</h2></html>')

'Some important text'

### Remove accented characters

In [17]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

remove_accented_chars('Sómě Áccěntěd těxt')

'Some Accented text'

### Expand contractions

Converting each contraction to its expanded, original form helps with text standardization

In [19]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions("Y'all can't expand contractions I'd think")

'You all cannot expand contractions I would think'

### Remove special characters

- Removing digits is optional, because often we might need to keep them in the pre-processed text.

In [20]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

remove_special_characters("Well this was fun! What do you think? 123#@!", 
                          remove_digits=True)

'Well this was fun What do you think '

### Text lemmatization

- Very similar to stemming, where we remove word affixes to get to the base form of a word.

    - The difference being that the **root word is always a lexicographically correct word** (present in the dictionary)

- Both nltk and spacy have excellent lemmatizers. We will be using spacy here.

In [21]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

lemmatize_text("My system keeps crashing! his crashed yesterday, ours crashes daily")

'My system keep crash ! his crashed yesterday , ours crash daily'

### Text stemming

- Word stems are also known as the **base form** of a word, and we can create new words by attaching affixes to them in a process 

    - Consider the word **JUMP**: JUMPS, JUMPED, JUMPING 
    
- Stemming helps us in standardizing words to their base or root stem, irrespective of their inflections
    - helps many applications like classifying or clustering text, and even in information retrieval

In [22]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

simple_stemmer("My system keeps crashing his crashed yesterday, ours crashes daily")

'My system keep crash hi crash yesterday, our crash daili'

### Remove stopwords

- Words which have little or no significance, especially when constructing meaningful features from text

- There is no universal stopword list, but we use a standard English language stopwords list from nltk. You can also add your own domain-specific stopwords as needed.


In [23]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

remove_stopwords("The, and, if are stopwords, computer is not")

', , stopwords , computer not'

### Bringing it all together, building a text normalizer

- We can keep going with more techniques like correcting spelling, grammar and so on

- At the end we have to chain these operations to build a text normalizer to pre-process text data.

In [24]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

#### Pre-process and normalize news articles

In [25]:
# combining headline and article text
news_df['full_text'] = news_df["news_headline"].map(str)+ '. ' + news_df["news_article"]

In [26]:
# pre-process text and store the same
news_df['clean_text'] = normalize_corpus(news_df['full_text'])
norm_corpus = list(news_df['clean_text'])

# show a sample news article
news_df.iloc[1][['full_text', 'clean_text']].to_dict()

{'full_text': "New TVS Ntorq 125 scooter features Bluetooth & smart connectivity. The TVS Ntorq 125, India's first scooter comes with Bluetooth support and smart connectivity features. It takes you to places with Navigation Assist and keeps you connected all the time with Caller ID and SMS alerts. Featuring SmartXonnect technology and sporty design, TVS Ntorq 125's special edition will also feature a special 'Scooter of the Year' insignia on the front.\n",
 'clean_text': 'new tvs ntorq scooter feature bluetooth smart connectivity tvs ntorq india first scooter come bluetooth support smart connectivity feature take place navigation assist keep connect time caller would sms alert feature smartxonnect technology sporty design tvs ntorq special edition also feature special scooter year insignia front'}

#### Save this dataset to disk if needed, so that you can always load it up later for future analysis.

In [27]:
news_df.to_csv('news.csv', index=False, encoding='utf-8')

## Understanding Language Syntax and Structure

For any language, syntax and structure usually go hand in hand, where a set of specific rules, conventions, and principles govern the way:

- words are combined into phrases 
- phrases get combines into clauses
- clauses get combined into sentences

Knowledge about the structure and syntax of language is helpful in many areas like text processing, annotation, and parsing for further operations such as text classification or summarization.

Typical parsing techniques for understanding text syntax:

- **Parts of Speech (POS) Tagging**
- **Shallow Parsing or Chunking**
- **Constituency Parsing**
- **Dependency Parsing**


## 3. Parts of Speech Tagging

Parts of speech (POS) are specific lexical categories to which words are assigned, based on their syntactic context and role. 

Usually, words can fall into one of the following major categories:

- N(oun): This usually denotes words that depict some object or entity, which may be living or nonliving. 

- V(erb): Verbs are words that are used to describe certain actions, states, or occurrences. There are a wide variety of further subcategories, such as auxiliary, reflexive, and transitive verbs (and many more).

- Adj(ective): Adjectives are words used to describe or qualify other words, typically nouns and noun phrases. 

- Adv(erb): Adverbs usually act as modifiers for other words including nouns, adjectives, verbs, or other adverbs.

Furthermore, each POS tag like the noun (N) can be further subdivided into categories like singular nouns (NN), singular proper nouns (NNP), and plural nouns (NNS).

The process of classifying and labeling POS tags for words called *parts of speech tagging* or *POS tagging* . 

![title](media/pos-tagging.png)

In [29]:
news_df = pd.read_csv('news.csv')

In [33]:
# create a basic pre-processed corpus, don't lowercase to get POS context
corpus = normalize_corpus(news_df['full_text'], text_lower_case=False, 
                          text_lemmatization=False, special_char_removal=False)

# demo for POS tagging for sample news headline
sentence = str(news_df.iloc[1].news_headline)
sentence_nlp = nlp(sentence)

In [34]:
# POS tagging with Spacy
spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in sentence_nlp]
pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])

,Word,POS tag,Tag type
0,New,JJ,ADJ
1,TVS,NNP,PROPN
2,Ntorq,NNP,PROPN
3,125,CD,NUM
4,scooter,NN,NOUN
5,features,VBZ,VERB
6,Bluetooth,NNP,PROPN
7,&,CC,CCONJ
8,smart,JJ,ADJ
9,connectivity,NN,NOUN


In [37]:
# POS tagging with nltk
nltk_pos_tagged = nltk.pos_tag(sentence.split())
pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])

,Word,POS tag
0,New,NNP
1,TVS,NNP
2,Ntorq,NNP
3,125,CD
4,scooter,NN
5,features,NNS
6,Bluetooth,NNP
7,&,CC
8,smart,JJ
9,connectivity,NN


## 4. Shallow Parsing or Chunking Text

Also known as *light parsing* or *chunking* , is a popular natural language processing technique of analyzing the structure of a sentence to break it down into its smallest constituents (which are tokens such as words) and group them together into higher-level phrases.

It includes POS tags as well as phrases from a sentence.

There are five major categories of phrases:

- Noun phrase (NP): These are phrases where a noun acts as the head word. Noun phrases act as a subject or object to a verb.

- Verb phrase (VP): These phrases are lexical units that have a verb acting as the head word. Usually, there are two forms of verb phrases. One form has the verb components as well as other entities such as nouns, adjectives, or adverbs as parts of the object.

- Adjective phrase (ADJP): These are phrases with an adjective as the head word. Their main role is to describe or qualify nouns and pronouns in a sentence, and they will be either placed before or after the noun or pronoun.

- Adverb phrase (ADVP): These phrases act like adverbs since the adverb acts as the head word in the phrase. Adverb phrases are used as modifiers for nouns, verbs, or adverbs themselves by providing further details that describe or qualify them.

- Prepositional phrase (PP): These phrases usually contain a preposition as the head word and other lexical components like nouns, pronouns, and so on. These act like an adjective or adverb describing other words or phrases.

![title](media/shallow-parsing.png)

We will leverage the conll2000 corpus for training our shallow parser model. This corpus is available in nltk with chunk annotations and we will be using around 10K records for training our model.

In [40]:
from nltk.corpus import conll2000

data = conll2000.chunked_sents()
train_data = data[:10900]
test_data = data[10900:] 

print(len(train_data), len(test_data))
print(train_data[1]) 

10900 48
(S
  Chancellor/NNP
  (PP of/IN)
  (NP the/DT Exchequer/NNP)
  (NP Nigel/NNP Lawson/NNP)
  (NP 's/POS restated/VBN commitment/NN)
  (PP to/TO)
  (NP a/DT firm/NN monetary/JJ policy/NN)
  (VP has/VBZ helped/VBN to/TO prevent/VB)
  (NP a/DT freefall/NN)
  (PP in/IN)
  (NP sterling/NN)
  (PP over/IN)
  (NP the/DT past/JJ week/NN)
  ./.)


Our data points are sentences that are already annotated with phrases and POS tags metadata that will be useful in training our shallow parser model.

Next, we will leverage two chunking utility functions, tree2conlltags , to get triples of word, tag, and chunk tags for each token, and conlltags2tree to generate a parse tree from these token triples. 

We will be using these functions to train our parser. 

In [41]:
from nltk.chunk.util import tree2conlltags, conlltags2tree

wtc = tree2conlltags(train_data[1])
wtc

[('Chancellor', 'NNP', 'O'),
 ('of', 'IN', 'B-PP'),
 ('the', 'DT', 'B-NP'),
 ('Exchequer', 'NNP', 'I-NP'),
 ('Nigel', 'NNP', 'B-NP'),
 ('Lawson', 'NNP', 'I-NP'),
 ("'s", 'POS', 'B-NP'),
 ('restated', 'VBN', 'I-NP'),
 ('commitment', 'NN', 'I-NP'),
 ('to', 'TO', 'B-PP'),
 ('a', 'DT', 'B-NP'),
 ('firm', 'NN', 'I-NP'),
 ('monetary', 'JJ', 'I-NP'),
 ('policy', 'NN', 'I-NP'),
 ('has', 'VBZ', 'B-VP'),
 ('helped', 'VBN', 'I-VP'),
 ('to', 'TO', 'I-VP'),
 ('prevent', 'VB', 'I-VP'),
 ('a', 'DT', 'B-NP'),
 ('freefall', 'NN', 'I-NP'),
 ('in', 'IN', 'B-PP'),
 ('sterling', 'NN', 'B-NP'),
 ('over', 'IN', 'B-PP'),
 ('the', 'DT', 'B-NP'),
 ('past', 'JJ', 'I-NP'),
 ('week', 'NN', 'I-NP'),
 ('.', '.', 'O')]

We will now define a function conll_tag_ chunks() to extract POS and chunk tags from sentences with chunked annotations and a function called combined_taggers() to train multiple taggers with backoff taggers (e.g. unigram and bigram taggers)

In [42]:
def conll_tag_chunks(chunk_sents):
    tagged_sents = [tree2conlltags(tree) for tree in chunk_sents]
    return [[(t, c) for (w, t, c) in sent] for sent in tagged_sents]


def combined_tagger(train_data, taggers, backoff=None):
    for tagger in taggers:
        backoff = tagger(train_data, backoff=backoff)
    return backoff 

We will now define a class NGramTagChunker that will take in tagged sentences as training input, get their (word, POS tag, Chunk tag) WTC triples, and train a BigramTagger with a UnigramTagger as the backoff tagger. 

We will also define a parse() function to perform shallow parsing on new sentences

We will use this class to train on the conll2000 chunked train_data and evaluate the model performance on the test_data

In [43]:
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk import ChunkParserI

# define the chunker class
class NGramTagChunker(ChunkParserI):
    
  def __init__(self, train_sentences, 
               tagger_classes=[UnigramTagger, BigramTagger]):
    train_sent_tags = conll_tag_chunks(train_sentences)
    self.chunk_tagger = combined_tagger(train_sent_tags, tagger_classes)

  def parse(self, tagged_sentence):
    if not tagged_sentence: 
        return None
    pos_tags = [tag for word, tag in tagged_sentence]
    chunk_pos_tags = self.chunk_tagger.tag(pos_tags)
    chunk_tags = [chunk_tag for (pos_tag, chunk_tag) in chunk_pos_tags]
    wpc_tags = [(word, pos_tag, chunk_tag) for ((word, pos_tag), chunk_tag)
                     in zip(tagged_sentence, chunk_tags)]
    return conlltags2tree(wpc_tags)
  
# train chunker model  
ntc = NGramTagChunker(train_data)

# evaluate chunker model performance
print(ntc.evaluate(test_data))

ChunkParse score:
    IOB Accuracy:  90.0%%
    Precision:     82.1%%
    Recall:        86.3%%
    F-Measure:     84.1%%


Our chunking model gets an accuracy of around 90% which is quite good

Let’s now leverage this model to shallow parse and chunk our sample news article headline which we used earlier

In [45]:
chunk_tree = ntc.parse(nltk_pos_tagged)
print(chunk_tree)

(S
  (NP
    New/NNP
    TVS/NNP
    Ntorq/NNP
    125/CD
    scooter/NN
    features/NNS
    Bluetooth/NNP)
  &/CC
  (NP smart/JJ connectivity/NN))
